In [1]:
# libraries
import pandas as pd
import numpy as np
import folium
from folium import plugins as fol_pi
import ipywidgets as widgets
from ipywidgets import interact
import IPython.display as IPdisp


In [2]:
def Transmission_score_formula(x, y, z):
    return 2*(x+y+z)

def Access_score_formula(x, y):
    if x < 3:
        return 2*y
    elif x < 5:
        return 1 + 2*y
    else:
        return 2 + 2*y

def Herd_size_score(herd):
    if herd < 1:
        return 0
    elif herd <10:
        return 1
    elif herd < 50:
        return 2
    elif herd < 100:
        return 3
    else:
        return 4

def Disease_score_formula(c_herd, b_herd, g_herd, c_dis, b_dis, g_dis):
    return (Herd_size_score(c_herd)+Herd_size_score(b_herd)+Herd_size_score(g_herd)+c_dis+b_dis+g_dis)/3

def Use_score_formula(u, v, w, x, y, z):
    return (u + v + w + x + y + z)

def Overall_score_formula(tra, acc, dis, use):
    return (tra + acc + dis + use)

def base_AMR_metr(manip_amr_data):
    manip_amr_data['Transmission score'] = manip_amr_data.apply(lambda row: Transmission_score_formula(row['Spreading dung on fields'], row['Not cleaning with disinfectant'], row['Visited by animal healthcare worker']), axis=1)        
    manip_amr_data['Access score'] = manip_amr_data.apply(lambda row: Access_score_formula(row['Frequency of antibiotic purchase'], row['Use of trained animal healthcare professional']), axis=1)        
    manip_amr_data['Disease score'] = manip_amr_data.apply(lambda row: Disease_score_formula(row['Cattle herd size'], row['Buffalo herd size'], row['Goat herd size'], row['Disease priority cattle'], row['Disease priority buffalo'], row['Disease priority goats']), axis=1)        
    manip_amr_data['Use score'] = manip_amr_data.apply(lambda row: Use_score_formula(row['Mixing antibiotics in the same animal'], row['Not completing course'], row['Not measuring dose accurately'], row['Using for prevention'], row['Drink milk from treated animals'], row['Eat meat from treated animals']), axis=1)
    # Now four component scores are combined for final AMR Metric
    manip_amr_data['Overall score'] = manip_amr_data.apply(lambda row: Overall_score_formula(row['Transmission score'], row['Access score'], row['Disease score'], row['Use score']), axis=1)
    return manip_amr_data

#manip_amr_data_community['Overall score'] = manip_amr_data_community.apply(lambda row: Overall_score_formula(row['Transmission score'], row['Access score'], row['Disease score'], row['Use score']), axis=1)


# NEXT scores are aggregated at community level, only necessary variables are kept
#Group by works fine, almost instant
# manip_amr_data_community = manip_amr_data.groupby(['Community']).mean()

# amr_metric_community = manip_amr_data_community[['Latitude','Longitude','Transmission score','Access score','Disease score','Use score','Overall score']]

#set ID as index for visual display purposes
# manip_amr_data.set_index('ID', inplace=True)

#amr_metric_community
#manip_amr_data

## In the below we build the user interactivity functions, to be manipulated using widgets

In [3]:
def user_Transmission_score_formula(dung_field, shed_clean, health_visit, add_dung, add_clean, add_visit):
    return max(0,min(2*(dung_field+add_dung),2)) + max(0,min(2*(shed_clean+add_clean),2)) + max(0,min(2*(health_visit+add_visit),2)) #to be applied at community level

def user_Access_score_formula(freq_purch, trained_risk, add_freq, add_trained):
    risk = max(0,min(trained_risk + add_trained,2))
    freq = freq_purch + add_freq
    if freq < 3:
        return 0 + 2*risk
    elif freq < 5:
        return 1 + 2*risk
    else:
        return 2 + 2*risk

def user_Disease_score_formula(c_herd, b_herd, g_herd, c_dis, b_dis, g_dis,\
                               add_c_herd, add_b_herd, add_g_herd,\
                              add_c_dis, add_b_dis, add_g_dis):
    user_c_dis = min(2,max(0,c_dis + add_c_dis))
    user_b_dis = min(2,max(0,b_dis + add_b_dis))
    user_g_dis = min(2,max(0,g_dis + add_g_dis))    
    return (Herd_size_score(c_herd+add_c_herd)+Herd_size_score(b_herd+add_b_herd)+Herd_size_score(g_herd+add_g_herd)+\
            user_c_dis+user_b_dis+user_g_dis)/3


def user_Use_score_formula(mix_anti, not_comp, not_meas, use_prev, drink_mlk, eat_meat,\
                          add_mix_anti, add_not_comp, add_not_meas, add_use_prev, add_drink_mlk, add_eat_meat):
    return max(0,min((mix_anti+add_mix_anti),1)) + max(0,min((not_comp+add_not_comp),1)) + max(0,min((not_meas+add_not_meas),1))\
+ max(0,min((use_prev+add_use_prev),1)) + max(0,min((drink_mlk+add_drink_mlk),1)) + max(0,min((eat_meat+add_eat_meat),1))

#to be applied at community level


def user_Overall_score_formula(tra, acc, dis, use):
    return (tra + acc + dis + use)
                                                                                    
                                                                                    
# Next we apply Access and Disease score formula at household level, then take the mean to allow Transmission and Use scored to be manipulated by user

def user_AMR_comp(amr_input_data, add_dung, add_clean,\
                  add_visit, add_freq, add_trained,\
                 add_c_herd, add_b_herd, add_g_herd,\
                  add_c_dis, add_b_dis, add_g_dis,\
                 add_mix_anti, add_not_comp, add_not_meas,\
                  add_use_prev, add_drink_mlk, add_eat_meat):
    
    #first apply the household level user manipulation scores
    amr_input_data['Access score'] = amr_input_data.apply(lambda row: user_Access_score_formula(row['Frequency of antibiotic purchase'], row['Use of trained animal healthcare professional'], add_freq, add_trained), axis=1)        
    amr_input_data['Disease score'] = amr_input_data.apply(lambda row: user_Disease_score_formula(row['Cattle herd size'], row['Buffalo herd size'], row['Goat herd size'], row['Disease priority cattle'], row['Disease priority buffalo'], row['Disease priority goats'], add_c_herd, add_b_herd, add_g_herd, add_c_dis, add_b_dis, add_g_dis), axis=1)        
    #next groupby to community
    manip_amr_data_community = amr_input_data.groupby(['Community']).mean()
    
    #now apply two community level scores
    manip_amr_data_community['Transmission score'] = manip_amr_data_community.apply(lambda row: user_Transmission_score_formula(row['Spreading dung on fields'], row['Not cleaning with disinfectant'], row['Visited by animal healthcare worker'], add_dung, add_clean, add_visit), axis=1)        
    manip_amr_data_community['Use score'] = manip_amr_data_community.apply(lambda row: user_Use_score_formula(row['Mixing antibiotics in the same animal'], row['Not completing course'], row['Not measuring dose accurately'], row['Using for prevention'], row['Drink milk from treated animals'], row['Eat meat from treated animals'], add_mix_anti, add_not_comp, add_not_meas, add_use_prev, add_drink_mlk, add_eat_meat), axis=1)                                                                                     
    
                                                                                    
    # All four components scored, we can find the overall                                                                                
    manip_amr_data_community['Overall score'] = manip_amr_data_community.apply(lambda row: user_Overall_score_formula(row['Transmission score'], row['Access score'], row['Disease score'], row['Use score']), axis=1)
    
    return manip_amr_data_community



In [4]:
# create colour rules for each rag score
def colour_rag_component(val):
    colour = 'red'
    if type(val) is not str and val<=4:
        colour = 'orange'
    if type(val) is not str and val<2:
        colour = 'green'
    return 'color: %s' % colour

def colour_rag_overall(val):
    colour = 'red'
    if type(val) is not str and val<=18:
        colour = 'orange'
    if type(val) is not str and val<8:
        colour = 'green'
    return 'color: %s' % colour

# Allow zoom for table view
def magnify():
    return [dict(selector="th", #TableHeaders
                 props=[("font-size", "11pt")]),
            dict(selector="td", #TableData
                 props=[('padding', "0em 3em"),
                        ('font-size', '10pt')]),
            dict(selector="th:hover",
                 props=[("font-size", "14pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '14pt')])]

# Adapt for map

def colour_rag_overall_marker(val):
    colour = 'red'
    if type(val) is not str and val<=18:
        colour = 'orange'
    if type(val) is not str and val<8:
        colour = 'green'
    return colour


In [5]:
def create_map(user_AMR_data, lat, lon, tiles="OpenStreetMap", zoom=13):
    m = folium.Map(location=(lat, lon), tiles=tiles, zoom_start=zoom)    

# amr_metric_community = manip_amr_data_community[['Latitude','Longitude','Transmission score','Access score','Disease score','Use score','Overall score']]
    
    for index, row in user_AMR_data.iterrows():
        #print(index, row['village'], row['Latitude'], row['Longitude'])
        html="""
        <h1>Community: {com}</h1>
        <p style="{col_ov};"><strong>Overall score {ov:.2f}</strong></p>
        <p style="{col_tr};">Transmission score: {tr:.2f}</p>
        <p style="{col_ac};">Access score: {ac:.2f}</p>
        <p style="{col_di};">Disease score: {di:.2f}</p>
        <p style="{col_us};">Use score: {us:.2f}</p>
        """.format(com=index,ov=row['Overall score'],col_ov=colour_rag_overall(row['Overall score']),\
                   tr=row['Transmission score'],col_tr=colour_rag_component(row['Transmission score']),\
                   ac=row['Access score'],col_ac=colour_rag_component(row['Access score']),\
                   di=row['Disease score'],col_di=colour_rag_component(row['Disease score']),\
                   us=row['Use score'],col_us=colour_rag_component(row['Use score']))

        iframe = folium.IFrame(html=html, width=300, height=300)
        popup = folium.Popup(iframe, parse_html=True)#, max_width=2650)

        folium.Marker([row['Latitude'],row['Longitude']],popup=popup, icon = folium.Icon(color=colour_rag_overall_marker(row['Overall score']),icon='fire',prefix='fa',icon_color='aquamarine')).add_to(m)
        # Note - icon colors restricted - see help(folium.Icon), icon_colour accepts HTML - see https://www.rapidtables.com/web/color/html-color-codes.html
        # Note - for icons see https://fontawesome.com/icons?d=gallery&m=free (not all work)
    

    m.add_child(folium.LatLngPopup())#, folium.LayerControl().add_to(m)
    
    fol_pi.Fullscreen(
    position='topright',
    title='Full screen',
    title_cancel='Exit full screen',
    force_separate_button=True).add_to(m)
    
    return m


Output()

In [6]:
sample_locations = ['Please select region to display map and data:','Gujarat, India', 'Pune, India', 'Kajiado, Kenya']

def AMR_tool(input_data='Gujarat, India', add_dung_widg = 0, add_clean_widg = 0, add_visit_widg = 0,\
             add_freq_widg=0, add_trained_widg=0,\
            add_c_herd_widg=0, add_b_herd_widg=0, add_g_herd_widg=0,\
             add_c_dis_widg=0, add_b_dis_widg=0, add_g_dis_widg=0,\
             add_mix_anti_widg=0, add_not_comp_widg=0, add_not_meas_widg=0,\
             add_use_prev_widg=0, add_drink_mlk_widg=0, add_eat_meat_widg=0):
    
    if input_data == 'Gujarat, India':
        amr_input_data = pd.read_csv("data/gujarat_csv_data.csv") #load csv
        base_AMR_data = base_AMR_metr(amr_input_data) #calculate base AMR metric on all households
        
        lat_mean = (base_AMR_data['Latitude'].min() + base_AMR_data['Latitude'].max())/2
        lon_mean = (base_AMR_data['Longitude'].min() + base_AMR_data['Longitude'].max())/2
    else:
        return
        
    # Generate user manipulated data
    user_AMR_data = user_AMR_comp(amr_input_data=amr_input_data,\
                                  add_dung=add_dung_widg, add_clean=add_clean_widg, add_visit=add_visit_widg,\
                                  add_freq=add_freq_widg, add_trained=add_trained_widg,\
                                  add_c_herd = add_c_herd_widg, add_b_herd = add_b_herd_widg, add_g_herd = add_g_herd_widg,\
                                  add_c_dis = add_c_dis_widg, add_b_dis = add_b_dis_widg, add_g_dis = add_g_dis_widg,\
                                  add_mix_anti = add_mix_anti_widg, add_not_comp = add_not_comp_widg, add_not_meas = add_not_meas_widg,\
                                  add_use_prev = add_use_prev_widg, add_drink_mlk = add_drink_mlk_widg, add_eat_meat = add_eat_meat_widg)
        
    # Show user manipulated data on map
    m = create_map(user_AMR_data=user_AMR_data, lat=lat_mean, lon=lon_mean, tiles="OpenStreetMap", zoom=13)
    return IPdisp.display(m)

# Build widgets

title = widgets.HTML(value="<h1 style='font-size:40px;background-color:LightBlue;text-align:center;border:2px solid Black;'>\
    AMR Data Exploration Tool <a style='color:Orange;'>(beta v0.1)</a></h1> \
    <p style='font-size:20px;text-align:center;'>Optional text here - e.g. Communities from region selected are shown on map, data tables provided below</p>",\
                     layout=widgets.Layout(width='100%', height='200%'))


input_data = widgets.Dropdown(options=sample_locations)
add_dung_widg = widgets.FloatSlider(value=0,min=-1,max=1,step=0.05,description='Spreading dung on fields:',readout_format='+.0%',continuous_update=False)
add_clean_widg = widgets.FloatSlider(value=0,min=-1,max=1,step=0.05,description='Not cleaning shed with disinfectant:',readout_format='+.0%',continuous_update=False)
add_visit_widg = widgets.FloatSlider(value=0,min=-1,max=1,step=0.05,description='Visited by animal healthcare worker:',readout_format='+.0%',continuous_update=False)

add_trained_widg = widgets.FloatSlider(value=0,min=-2,max=2,step=1,description='Use of trained animal healthcare professional:',readout_format='+d',continuous_update=False)
add_freq_widg = widgets.BoundedIntText(value=+0,min=-100,max=100,step=1,description='+/- Frequency of antibiotic purchase:',continuous_update=False)

add_c_herd_widg = widgets.BoundedIntText(value=0,min=-100,max=100,step=10,description='+/- Cattle herd size:',continuous_update=False,style = {'description_width': 'initial'})
add_b_herd_widg = widgets.BoundedIntText(value=0,min=-100,max=100,step=10,description='+/- Buffalo herd size:',continuous_update=False,style = {'description_width': 'initial'})
add_g_herd_widg = widgets.BoundedIntText(value=0,min=-100,max=100,step=10,description='+/- Goat herd size:',continuous_update=False,style = {'description_width': 'initial'})

add_c_dis_widg = widgets.FloatSlider(value=0,min=-2,max=2,step=1,description='Cattle disease risk household level:',readout_format='+d',continuous_update=False)
add_b_dis_widg = widgets.FloatSlider(value=0,min=-2,max=2,step=1,description='Buffalo disease risk household level:',readout_format='+d',continuous_update=False)
add_g_dis_widg = widgets.FloatSlider(value=0,min=-2,max=2,step=1,description='Goat disease risk household level:',readout_format='+d',continuous_update=False)

add_mix_anti_widg = widgets.FloatSlider(value=0,min=-1,max=1,step=0.05,description='Mixing antibiotics in the same animal:',readout_format='+.0%',continuous_update=False)
add_not_comp_widg = widgets.FloatSlider(value=0,min=-1,max=1,step=0.05,description='Not completing course:',readout_format='+.0%',continuous_update=False)
add_not_meas_widg = widgets.FloatSlider(value=0,min=-1,max=1,step=0.05,description='Not measuring dose accurately:',readout_format='+.0%',continuous_update=False)
add_use_prev_widg = widgets.FloatSlider(value=0,min=-1,max=1,step=0.05,description='Using for prevention:',readout_format='+.0%',continuous_update=False)

add_drink_mlk_widg = widgets.FloatSlider(value=0,min=-1,max=1,step=0.05,description='Drink milk from treated animals:',readout_format='+.0%',continuous_update=False)
add_eat_meat_widg = widgets.FloatSlider(value=0,min=-1,max=1,step=0.05,description='Eat meat from treated animals:',readout_format='+.0%',continuous_update=False)



# Layout widgets

title_box = widgets.HBox([title])

in_box = widgets.HBox([input_data])

tran_box = widgets.HBox([add_dung_widg, add_clean_widg, add_visit_widg])

access_box = widgets.HBox([add_trained_widg,add_freq_widg])

herd_size_box = widgets.HBox([add_c_herd_widg,add_b_herd_widg,add_g_herd_widg])
herd_dis_box = widgets.HBox([add_c_dis_widg,add_b_dis_widg,add_g_dis_widg])

disease_box = widgets.VBox([herd_size_box,herd_dis_box])

use_anti_box = widgets.HBox([add_mix_anti_widg,add_not_comp_widg,add_not_meas_widg])
use_risk_box = widgets.HBox([add_use_prev_widg,add_drink_mlk_widg,add_eat_meat_widg])

use_box = widgets.VBox([use_anti_box,use_risk_box])

accordion = widgets.Accordion(children=[tran_box, access_box, disease_box, use_box], selected_index=None)
accordion.set_title(0, 'Transmission component data interaction (+/- proportion of "Yes" in communities by variable)')
accordion.set_title(1, 'Access component data interaction (+/- "Use of trained professional risk level" and "Antibiotic purchase frequency" per household)')
accordion.set_title(2, 'Disease component data interaction (+/- "Herd size" and "Disease risk level" by animal per household)')
accordion.set_title(3, 'Use component data interaction (+/- proportion of "Yes" in communities by variable)')
#ui = widgets.VBox([title_box, in_box,tran_box,access_box,herd_size_box,herd_dis_box,use_anti_box,use_risk_box])

ui = widgets.VBox([title_box, in_box,accordion])

out = widgets.interactive_output(AMR_tool, {'input_data': input_data,\
                                            'add_dung_widg': add_dung_widg,'add_clean_widg': add_clean_widg,'add_visit_widg': add_visit_widg,\
                                            'add_freq_widg': add_freq_widg, 'add_trained_widg': add_trained_widg,\
                                           'add_c_herd_widg':add_c_herd_widg,'add_b_herd_widg':add_b_herd_widg,'add_g_herd_widg':add_g_herd_widg,\
                                           'add_c_dis_widg':add_c_dis_widg,'add_b_dis_widg':add_b_dis_widg,'add_g_dis_widg':add_g_dis_widg,\
                                           'add_mix_anti_widg':add_mix_anti_widg,'add_not_comp_widg':add_not_comp_widg,'add_not_meas_widg':add_not_meas_widg,\
                                           'add_use_prev_widg':add_use_prev_widg,'add_drink_mlk_widg':add_drink_mlk_widg,'add_eat_meat_widg':add_eat_meat_widg})

display(ui, out)

Output()